## Hybrid Search
 ### Semanic Search
      
      Dence Vector Search--->Similar

 ### syntatic Search 
      
      Exact Search-->keyword search-->parse vector search



Sparce metrix-->if we converted embeding with ---> One hot encoding--->Bag of words--->TFIDF--->Text--->Parse matrix


0001000<---- Saprce metrix --->one is defined as keyword

how we can performe hybrid seach :


semantic search and  syntatic search combine bothe result based on ranking of resul of bothe emebdings 


time score=sumation (1/(c+Rd))

c-Dependent on verious Database
Rd-Rank of d(document)
1-->60

=sumation(1/Rd)



### Graph Knowledge 

 it support -->Kyeword Search-->Semantic Search-->Graph Knoledge Search



In [14]:
!pip install --upgrade --quiet pinecone pinecone-text pinecone-notebooks

In [15]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [21]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## Initialize the Pinecone Client
os.environ['PINECONE_API_KEY'] ='pcsk_4Ui4M6_NKUtFqRF8nPW1DQDFCPrMTgoE5D4fNeGiCjjVLN4moRMopLQnp4FVGbr4DU8CJH'
pc=Pinecone(api_key=os.getenv("PINECONE_API_KEY"))



# create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, #dimention of dens metrix vector
        metric='dotproduct' ,# sparse value supportted only for dotproduct
        spec=ServerlessSpec(cloud='aws',region='us-east-1'),
    )

In [23]:
index=pc.Index(index_name)
index

In [25]:
!pip install sentence-transformers

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   -------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.22.1 which is incompatible.
langchain-chroma 0.2.6 requires chromadb>=1.0.20, but you have chromadb 0.5.23 which is incompatible.


In [28]:
### vectore embedding and sparce matrix


import os
from dotenv import load_dotenv
load_dotenv()
os.environ["HF_token"]=os.getenv("HF_token")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [29]:
from pinecone_text.sparse import BM25Encoder
bs25_encoder=BM25Encoder().default()
bs25_encoder

In [30]:
sentence=[
    "in 2023 ,I visited paris",
    "In 2022, I visited New York",
    "In 2021,I visited New Orleans",
]
## TFIDF values on the sentence
bs25_encoder.fit(sentence)

# store the values to a json file
bs25_encoder.dump("mb25_values_json")

# load to your BM25ENCODER object
bm25_encoder=BM25Encoder().load("mb25_values_json")

100%|██████████| 3/3 [00:00<00:00, 17.14it/s]


In [31]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [32]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001B93791DBD0>, index=<pinecone.db_data.index.Index object at 0x000001B9158F7B90>)

In [34]:
retriever.add_texts(["in 2023 ,I visited paris",
    "In 2022, I visited New York",
    "In 2021,I visited New Orleans",])

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.30s/it]


In [36]:
retriever.invoke("what city did visit first")

[Document(metadata={'score': 0.204705447}, page_content='in 2023 ,I visited paris'),
 Document(metadata={'score': 0.201699868}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.225356236}, page_content='In 2021,I visited New Orleans')]